# Because we had observed a lot of skewness in data for different commodities and APMC's in Data Exploration
-  We need to prepare data first for seasonal analysis
-  Setting a minimum data frequency (example : yearly, quarterly,half yearly)
-  Finally we need to import this specific data for Seasonality Detection and Pricing of commodities 

In [1]:
import pandas as pd
import numpy as np

In [7]:
mandi_data = pd.read_csv("Outlier_removed_mandi_data.csv")

# Since, time-series is heavily based on calculations with 'DATE',we convert date column's data-type to enable arithmetic operations on data

In [10]:
mandi_data.drop(labels = ['Unnamed: 0'],inplace=True,axis=1)
mandi_data['date'] = pd.to_datetime(mandi_data['date'])

In [11]:
mandi_data.head()

,APMC,commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name
0,Kalyan (Cattle Market),kalvad,2016,February,3,12000,18000,15000.0,2016-02-01,Thane
1,Kalyan (Cattle Market),kalvad,2016,June,3,25000,30000,28000.0,2016-06-01,Thane
2,Kalyan,chandani,2015,December,6,6000,7000,6900.0,2015-12-01,Thane
3,Kalyan,chandani,2016,January,6,10000,12000,12000.0,2016-01-01,Thane
4,Kalyan,aster,2015,December,15,8000,9600,8860.0,2015-12-01,Thane


# The Data looks fine, now let's filter data such that it must have 12 values,
-  (All months data) for every unique combination of Year, APMC and commodity

In [19]:
prep_data = mandi_data[['APMC','commodity','Year','modal_price']].groupby(['APMC','Year','commodity'],
                        as_index=False).count().rename(columns={'modal_price':'Frequency'})

In [20]:
prep_data = prep_data[prep_data['Frequency']==12].reset_index(drop=True)

In [24]:
prep_data.head()

,APMC,Year,commodity,Frequency
0,Aamgaon,2015,paddy-unhusked,12
1,Akhadabalapur,2015,soybean,12
2,Akluj,2015,bitter gourd,12
3,Akluj,2015,brinjal,12
4,Akluj,2015,cabbage,12


# Addressing the number of Data values for seasonality Analysis
-  845 , after constarining value frequency

In [22]:
print(len(prep_data))

845


# Now choosing only those rows from the mandi_data, that contain combinations given in prep_data

In [48]:
final_df =pd.DataFrame()
for i,row in prep_data.iterrows():
    temp_df = mandi_data[(mandi_data['APMC']==row.APMC) & (mandi_data['commodity']==row.commodity)]
    final_df = pd.concat([temp_df,final_df])
final_df.reset_index(drop=True,inplace = True)    

In [49]:
final_df.head()

,APMC,commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name
0,Yeotmal,soybean,2015,April,12384,3311,3739,3527.0,2015-04-01,Yewatmal
1,Yeotmal,soybean,2016,April,20978,3661,4045,3857.0,2016-04-01,Yewatmal
2,Yeotmal,soybean,2015,August,5958,3029,3321,3240.0,2015-08-01,Yewatmal
3,Yeotmal,soybean,2016,August,3539,3089,3438,3282.0,2016-08-01,Yewatmal
4,Yeotmal,soybean,2014,December,18921,3014,3300,3163.0,2014-12-01,Yewatmal


# Now, finally we have the desired dataset, with frequency value of 12 for every unique combination of Year,Commodity and APMC

-  Finally, Exporting this dataset to be used while Analysis of Seasonality

In [51]:
final_df.to_csv("Seasonality_Analysis_data.csv",index=False)